https://www.kaggle.com/datasets/mohamedhanyyy/chest-ctscan-images/code?datasetId=839140&sortBy=voteCount

In [2]:

from tensorflow.keras.layers import Flatten, Dense, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential

In [3]:
# Define directories for training, validation, and test datasets



train_dir =  'path to augmented or basic training directory'
test_dir = 'path to basic testing directory' #only want to test on non-synthetic data
val_dir = 'path to augmented or basic validation directory' #its okay if validation data is augmented 


In [5]:
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import resnet

# Create separate ImageDataGenerator instances for each dataset. 
#Use resnet preprocess function because we use resnet 

train_datagen = ImageDataGenerator(preprocessing_function=resnet.preprocess_input)
val_datagen = ImageDataGenerator(preprocessing_function=resnet.preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=resnet.preprocess_input)

# Create separate generators for training, validation, and test datasets
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=20,
    class_mode='binary',  # Use 'categorical' for multi-class classification
    shuffle=True,
    seed=42
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=20,
    class_mode='binary',
    shuffle=True  
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=20,
    class_mode='binary',
    shuffle=True  
)

Found 621 images belonging to 2 classes.
Found 70 images belonging to 2 classes.
Found 144 images belonging to 2 classes.


In [6]:
img_size = (224, 224)
channels = 3
img_shape = (img_size[0], img_size[1], channels)
class_count = 2

In [7]:
from tensorflow.keras.applications import ResNet50
base_model = res_model = ResNet50(include_top=False, pooling='avg', weights='imagenet', input_shape = (img_shape)) #fine tuning! 

for layer in base_model.layers[:-2]: #Train the last 2 layers of resnet to improve accuracy. 
    layer.trainable = False 
    
model = Sequential([
    base_model,
    Flatten(),
    BatchNormalization(axis= -1, momentum= 0.99, epsilon= 0.001),
    Dense(224*224, kernel_regularizer= regularizers.l2(l= 0.016), activity_regularizer= regularizers.l1(0.006),
    bias_regularizer= regularizers.l1(0.006), activation= 'leaky_relu'),
    Dense(500, activation= 'leaky_relu'),
    Dense(200, activation= 'leaky_relu'), #leaky relu to avoid dying neurons 
    Dense(50, activation= 'leaky_relu'),
    Dense(1, activation= 'sigmoid') #one neuron output for binary 
])

model.compile(Adam(learning_rate= 0.005), loss= 'binary_crossentropy', metrics= ['accuracy'])

model.summary()

2023-12-11 14:57:07.983470: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 flatten (Flatten)           (None, 2048)              0         
                                                                 
 batch_normalization (BatchN  (None, 2048)             8192      
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 50176)             102810624 
                                                                 
 dense_1 (Dense)             (None, 500)               25088500  
                                                                 
 dense_2 (Dense)             (None, 200)               100200    
                                                        

In [8]:
#adjust learnign rate, train extra layer 

batch_size = 20
epochs = 5

patience = 3 		# number of epochs to wait to adjust lr if monitored value does not improve
stop_patience = 3 	# number of epochs to wait before stopping training if monitored value does not improve

from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint = ModelCheckpoint('model', monitor='val_loss', save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=stop_patience, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.0005, patience=patience, min_lr=1e-6)

history = model.fit(train_generator, validation_data=val_generator,
                    epochs= epochs, verbose= 1, shuffle= False,
                    callbacks=[checkpoint, early_stopping, reduce_lr],)

percentage = "x" #modify this depending on the percentage of synthetic data

model.save(f'/Users/benmaizes/Dropbox/Mac/Desktop/DIS/DIS_NN/FinalProject/data/.ipynb_checkpoints/{percentage}_x')



Epoch 1/5
32/32 [==============================] - ETA: 0s - loss: 475.6642 - accuracy: 0.8696

INFO:tensorflow:Assets written to: model/assets


INFO:tensorflow:Assets written to: model/assets


32/32 [==============================] - 188s 6s/step - loss: 475.6642 - accuracy: 0.8696 - val_loss: 656.5252 - val_accuracy: 0.7000 - lr: 0.0050
Epoch 2/5
32/32 [==============================] - ETA: 0s - loss: 472.0365 - accuracy: 0.9002

INFO:tensorflow:Assets written to: model/assets


...

INFO:tensorflow:Assets written to: model/assets


INFO:tensorflow:Assets written to: model/assets


32/32 [==============================] - 283s 9s/step - loss: 79.1010 - accuracy: 0.9485 - val_loss: 48.8110 - val_accuracy: 0.9857 - lr: 0.0050


INFO:tensorflow:Assets written to: /Users/benmaizes/Dropbox/Mac/Desktop/DIS/DIS_NN/FinalProject/data/.ipynb_checkpoints/augmented_x/assets


INFO:tensorflow:Assets written to: /Users/benmaizes/Dropbox/Mac/Desktop/DIS/DIS_NN/FinalProject/data/.ipynb_checkpoints/augmented_x/assets


In [9]:
# Evaluate the model on the test set using the test generator

test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test accuracy: {test_accuracy}")

8/8 [==============================] - 24s 3s/step - loss: 49.3420 - accuracy: 0.9861
Test accuracy: 0.9861111044883728
